# Terrain Visualization

In this activity, you will work with creating and manipulating 3D surface meshes using **PyVista**, a Python interface for the **Visualization Toolkit (VTK)**. VTK is a powerful open-source library for computer graphics, visualization, and image processing. You can learn more about both tools through these references:
- https://docs.pyvista.org/
- https://vtk.org/

We will also be using the **itkwidgets** library, which provides interactive Jupyter widgets for plotting, to visualize our meshes.

The outline of this activity will be:
1. Creating a 3D surface mesh
2. Writing code to coarsen the mesh
3. Writing code to visualize the error in elevation between the original mesh and the coarse mesh

# Part 1: Creating a 3D Surface Mesh
We will start by using a topographic surface to create a 3D terrain-following mesh.

Terrain following meshes are common in the environmental sciences, for instance
in hydrological modelling (see
[Maxwell 2013](https://www.sciencedirect.com/science/article/abs/pii/S0309170812002564)
and
[ParFlow](https://parflow.org)).

Below, we domonstrate a simple way to make a 3D grid/mesh that
follows a given topographic surface. In this example, it is important to note
that the given digital elevation model (DEM) is structured (gridded and not
triangulated): this is common for DEMs.


In [2]:
import math
import numpy as np
import pylab as plt
import pyvista as pv
from pyvista import examples

In [3]:
%matplotlib widget
from pyvista import set_plot_theme
set_plot_theme('document')
pv.set_jupyter_backend('pythreejs')

Download a gridded topography surface (DEM) using one of the examples provided by PyVista.



In [4]:
dem = examples.download_crater_topo()
dem

UniformGrid (0x7f987c196940)
  N Cells:	1677401
  N Points:	1680000
  X Bounds:	1.810e+06, 1.831e+06
  Y Bounds:	5.640e+06, 5.658e+06
  Z Bounds:	0.000e+00, 0.000e+00
  Dimensions:	1400, 1200, 1
  Spacing:	1.500e+01, 1.500e+01, 0.000e+00
  N Arrays:	1

In [5]:
#dir(dem)

In [6]:
type(dem)
print(dem.x)
print(dem.y)
print(dem.z)

[1810000. 1810015. 1810030. ... 1830955. 1830970. 1830985.]
[5640000. 5640000. 5640000. ... 5657985. 5657985. 5657985.]
[0. 0. 0. ... 0. 0. 0.]


In [7]:
#help(pv.UniformGridFilters.extract_subset)

Now let's subsample and extract an area of interest to make this example
simple (also the DEM we just loaded is pretty big).
Since the DEM we loaded is a `pyvista.UniformGrid` mesh, we can use
the `pyvista.UniformGridFilters.extract_subset` filter to extract a 257x257-point (256x256-cell) area from the DEM:



In [8]:
subset = dem.extract_subset((572, 828, 472, 728, 0, 0), (1,1,1))
subset
# type(subset)

UniformGrid (0x7f987c1201c0)
  N Cells:	65536
  N Points:	66049
  X Bounds:	1.819e+06, 1.822e+06
  Y Bounds:	5.647e+06, 5.651e+06
  Z Bounds:	0.000e+00, 0.000e+00
  Dimensions:	257, 257, 1
  Spacing:	1.500e+01, 1.500e+01, 0.000e+00
  N Arrays:	1

Let's plot the area we just extracted to see what it looks like.

In [9]:
pv.plot(subset)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

Now that we have a region of interest for our terrain following mesh, lets
make a 3D surface of that DEM.

In [10]:
terrain = subset.warp_by_scalar() #Warp into a 3D surface mesh (without volume)
terrain

StructuredGrid (0x7f987b6794c0)
  N Cells:	65536
  N Points:	66049
  X Bounds:	1.819e+06, 1.822e+06
  Y Bounds:	5.647e+06, 5.651e+06
  Z Bounds:	1.777e+03, 2.787e+03
  Dimensions:	257, 257, 1
  N Arrays:	1

We can see that our terrain is now a `pyvista.StructuredGrid` mesh. Now let's plot our terrain.

In [11]:
pv.plot(terrain)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Part 2: Coarsening the Mesh (and Writing Code)
In this section, you will write code to generate a new coarse mesh from our `terrain` mesh. Coarse meshes generally provide less accurate solutions, but are computationally faster. 

Your new mesh should be a `StructuredGrid`, just like the original mesh, but with a lower resolution. This means you will need to redefine the (x, y, z) coordinate points of your mesh. We will explain how to redefine your coordinates a little later on.

First, let's start with understanding how to generate a new mesh. You can initialize a new `StructuredGrid` object directly from the three point arrays that each contain the x, y, and z coordinates of all points in the mesh, respectively. Note: Each array is a 3D array with dimensions M x N x 1 (with the z-axis always being of length 1).

You will find the following reference helpful: https://docs.pyvista.org/version/stable/examples/00-load/create-explicit-structured-grid.html#sphx-glr-examples-00-load-create-explicit-structured-grid-py.

Let's look at the example below for initializing a new `StructuredGrid`.

In [12]:
xrng = np.arange(-10, 10, 2)                # [-10,  -8,  -6,  -4,  -2,   0,   2,   4,   6,   8]
yrng = np.arange(-10, 10, 2)
zrng = np.arange(-10, 10, 2)
x_example, y_example, z_example = np.meshgrid(xrng, yrng, zrng)
grid_example = pv.StructuredGrid(x_example, y_example, z_example)
grid_example

/opt/conda/lib/python3.8/site-packages/pyvista/utilities/helpers.py:475: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


StructuredGrid,Information
N Cells,729
N Points,1000
X Bounds,"-1.000e+01, 8.000e+00"
Y Bounds,"-1.000e+01, 8.000e+00"
Z Bounds,"-1.000e+01, 8.000e+00"
Dimensions,"10, 10, 10"
N Arrays,0


In [13]:
#dir(terrain)
# terrain.GetBounds()

In [14]:
# print(terrain.x.shape)
# new_x = terrain.x[::2,::2,:]
# print(new_x.shape)
# print(new_x[10,:,:])

In [15]:
# print(terrain.y.shape)
# new_y = terrain.y[::2,::2,:]
# print(new_y.shape)
# print(new_y[:,10,:])

In [16]:
print(terrain.z.shape)
new_z = terrain.z[0,:,0]
print(new_z)


(257, 257, 1)
[1804.57 1804.36 1803.23 1801.25 1798.36 1793.65 1790.99 1792.03 1795.19
 1797.06 1795.59 1789.51 1782.07 1777.38 1777.8  1783.47 1792.91 1801.77
 1810.81 1819.5  1826.43 1831.52 1834.17 1834.1  1832.09 1828.79 1826.46
 1825.62 1825.57 1825.77 1825.77 1825.21 1825.24 1828.46 1834.65 1839.69
 1842.16 1843.42 1843.53 1842.58 1840.67 1840.07 1840.19 1841.32 1844.54
 1847.05 1846.76 1845.26 1842.68 1841.07 1840.45 1840.19 1839.18 1834.99
 1828.16 1819.17 1808.71 1799.65 1792.3  1786.17 1781.34 1780.52 1783.19
 1786.13 1788.62 1790.18 1791.19 1791.82 1792.27 1792.69 1792.96 1791.83
 1788.14 1784.36 1780.55 1778.66 1779.96 1783.83 1788.96 1793.84 1797.72
 1801.39 1807.61 1814.21 1819.94 1822.76 1821.41 1819.23 1817.23 1820.5
 1827.57 1835.38 1837.32 1836.86 1836.73 1837.42 1837.27 1837.52 1837.06
 1836.15 1835.18 1834.97 1835.58 1835.9  1833.97 1830.76 1826.32 1821.2
 1817.71 1815.65 1816.17 1820.68 1828.6  1836.2  1839.94 1840.66 1840.09
 1840.3  1842.62 1844.41 1842.98 1839.9

Now, let's follow the same general steps as in the above example to generate our new coarse mesh from our previously created `terrain` mesh.

We can coarsen the mesh by merging every `f` quads/cells into one and dropping the center point, where `f` is your sampling factor aka the factor by which you want to reduce the resolution. In other words, we can produce a reduced version of the mesh by sampling one out of every `f` points along each axis of the mesh.

Write code to coarsen `terrain` by a **factor of 2**. In other words, we will be converting the mesh from a 257x257-point mesh to a 129x129-point mesh (or equivalently, a 256x256-cell mesh to a 128x128-cell mesh). 

In the code block below, complete the implementation of `coarsen(terrain)` by defining three new point arrays, `xnew`, `ynew`, and `znew` and composing them into a new `StructuredGrid` object named `coarse`. Your function should return a tuple containing `(xnew, ynew, znew, coarse)`.
You will find the following reference helpful: https://docs.pyvista.org/version/stable/api/core/_autosummary/pyvista.StructuredGrid.html.

In [17]:
#NOTE: You do not need to round any values within your results.
def coarsen(terrain, f=2):
   
    coarse_grid = pv.StructuredGrid(terrain.x[::f,::f,:], terrain.y[::f,::f,:], terrain.z[::f,::f,:])
    
    x_new = coarse_grid.x
    y_new = coarse_grid.y
    z_new = coarse_grid.z
    
    return(x_new, y_new, z_new, coarse_grid)
    
    ### END STUDENT SOLUTION

In [18]:
### Tests for coarsenMesh. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.

print(f"Coarsening from {terrain.dimensions[0]} to {math.ceil(terrain.dimensions[0]/2)}...")
xnew, ynew, znew, coarse = coarsen(terrain)

assert xnew.shape == (129,129,1)
assert ynew.shape == (129,129,1)
np.testing.assert_allclose(xnew[0][0][0],1818580, rtol=1e-7)
np.testing.assert_allclose(xnew[5][120][0],1818730, rtol=1e-7)
np.testing.assert_allclose(ynew[128][120][0],5650680, rtol=1e-7)
np.testing.assert_allclose(znew[12][12][0],1880.53, rtol=1e-5)

Coarsening from 257 to 129...


We can plot the z-values of our new coarsened mesh by adding an additional attribute `values` to our mesh, which will contain a normalized, column-major flattened representation of the z-axis values of our grid.

See the following reference for more information on array flattening: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html

In [19]:
#Plot the z-values using the viridis (default) color map
coarse['values'] = pv.plotting.normalize(coarse.z.flatten("F"))

pv.plot(coarse, scalars='values')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Part 3: Visualizing Error Values

Now that we have generated our coarse mesh, we can visualize the error in elevation between our coarse mesh and our original mesh. More specifically, we want to compute the error value for each point between the new (bilinearly interpolated) center point elevation and the original. We will then visualize the error as a scalar field on the original mesh.

Since we will need to bilinearly interpolate the center point elevation (i.e. the z-value) of each point in our coarse mesh in order to match the dimensions of our original mesh, let's define a function to do just that.

Define the function `bilin()` to bilinearly interpolate the value at coordinates `(x,y)` within a rectilinear grid of points.

**The parameters of your function are:**
- `x` = x-coordinate of point whose value we wish to interpolate
- `y` = y-coordinate of point whose value we wish to interpolate
- `points` = a list of four triplets of the form `(xc, yc, val)`, where `val` denotes the function value associated with coordinates `(xc, yc)`

This function should return a bilinearly interpolated value associated with coordinate `(x,y)` w.r.t the rectilinear grid formed by `points`.

**Hints:**
- You may assume the four triplets within `points` form a valid rectangle
- You may assume `x` and `y` fall within the rectangle formed by the `points` parameter
- You should NOT assume the four triplets within `points` are in any specific order

In [20]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
#NOTE: You do not need to round any values within your results.
def bilin(x, y, points):
    ### BEGIN STUDENT SOLUTION
    points_sorted = sorted(points, key=lambda x: x[0])
    g1 = sorted(points_sorted[:2], key=lambda x: x[1])  # Group One (left) sorted in y 
    g2 = sorted(points_sorted[2:], key=lambda x: x[1])  # Group Two (right) sorted in y
    
    # Interpolation in y
    t1 = (y - g1[0][1]) / (g1[1][1] - g1[0][1]) 
    v1 = (1 - t1) * g1[0][2] + t1 * g1[1][2] 
    v2 = (1 - t1) * g2[0][2] + t1 * g2[1][2]
    
    # Interpolation in x
    t2 = (x - g1[0][0]) / (g2[0][0] - g1[0][0])
    v_int = (1 - t2) * v1 + t2 * v2
    return v_int
    
    ### END STUDENT SOLUTION

In [21]:
# testing_points = [(1,1,3), (3,1,6), (1,3,7), (3,3,9)]
# v_int = bilin(2,2, testing_points)
# print(v_int)

In [22]:
### Tests for bilin(x, y, points) function. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.
testing_points = [(1,1,3), (3,1,6), (1,3,7), (3,3,9)]
result = bilin(2,2,testing_points)
np.testing.assert_allclose(result,6.25, rtol=1e-2)
result = bilin(2.5,2.5,testing_points)
np.testing.assert_allclose(result,7.6875, rtol=1e-3)
result = bilin(1.1,1.1,testing_points)
np.testing.assert_allclose(result,3.3475, rtol=1e-3)

Now, using your `bilin()` function, create a new mesh or `StructuredGrid` object named `intmesh`, reconstructed from `coarse` using bilinear interpolation, with the same dimensions as our original mesh `terrain`. Your new mesh should contain the interpolated z-values for each point in `terrain`.

As a starting point, we have defined some of the variables that will be helpful to you for creating your new interpolated mesh. Specifically, we will be checking the values in `errz` and `intz`, as defined below:
- `intz`: a 3D array with the same shape as `terrain.z` that will contain the bilinearly interpolated z-values from the coarsened mesh.<br/>**Note:** `intz` is a 3D M x N x 1 array where the last dimension contains the z-values. You should note this when assigning elements to `intz`. The interpolated mesh is a 3D M by N by 1 array, so tests will fail if you assign values to it as if it were just a 2D M by N array, bypassing the z-axis entirely. So, if your code looks like `intz[x][y] = bilin(...)` the fix is to change it to something like `intz[x][y] = [ bilin(...) ]` or  `intz[x][y][0] = bilin(...)`.

- `errz`: a list of scalar values. This should contain the absolute error values between each z-value in the original mesh and each interpolated z-value in the new returned mesh

Just like how we added the attribute `values` to our coarse mesh in order to plot the z-values of the mesh, you should add an additional attribute `errors` to `intmesh` in order to plot the absolute error values between the z-values in the original mesh and the interpolated z-values in our new returned mesh.

In the code block below, complete the implementation of `reconstructMesh(terrain,coarse)`, which should return a tuple containing `(intz, errz, intmesh)`.

In [23]:
np.zeros_like(terrain.z).shape

(257, 257, 1)

In [24]:
terrain.x[:-1,0,0]

pyvista_ndarray([1818580., 1818595., 1818610., 1818625., 1818640.,
                 1818655., 1818670., 1818685., 1818700., 1818715.,
                 1818730., 1818745., 1818760., 1818775., 1818790.,
                 1818805., 1818820., 1818835., 1818850., 1818865.,
                 1818880., 1818895., 1818910., 1818925., 1818940.,
                 1818955., 1818970., 1818985., 1819000., 1819015.,
                 1819030., 1819045., 1819060., 1819075., 1819090.,
                 1819105., 1819120., 1819135., 1819150., 1819165.,
                 1819180., 1819195., 1819210., 1819225., 1819240.,
                 1819255., 1819270., 1819285., 1819300., 1819315.,
                 1819330., 1819345., 1819360., 1819375., 1819390.,
                 1819405., 1819420., 1819435., 1819450., 1819465.,
                 1819480., 1819495., 1819510., 1819525., 1819540.,
                 1819555., 1819570., 1819585., 1819600., 1819615.,
                 1819630., 1819645., 1819660., 1819675., 18196

In [25]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
#NOTE: You do not need to round any values within your results.
import copy 
def reconstructMesh(terrain,coarse):
    #errz   = [] #Create a new empty list for holding absolute error values
    intz   = np.zeros_like(terrain.z) #Create a new array for holding bilinearly interpolated values from coarse mesh
    errz_mtx = np.zeros_like(terrain.z)
    scale  = (terrain.z.shape[0]-1)/(coarse.z.shape[0]-1) #Reduction factor between original and coarse; should equal 2

    ### BEGIN STUDENT SOLUTION
    x_vec_fine = list(terrain.x[:,0,0])
    y_vec_fine = list(terrain.y[0,:,0])
    
    x_vec_coarse = list(coarse.x[:,0,0])
    y_vec_coarse = list(coarse.y[0,:,0])
    
    for i in range(len(x_vec_coarse) - 1):
        for j in range(len(y_vec_coarse) - 1):
            points = [(x_vec_coarse[i], y_vec_coarse[j], coarse.z[i, j, 0]),
                     (x_vec_coarse[i + 1], y_vec_coarse[j], coarse.z[i + 1, j, 0]),
                     (x_vec_coarse[i], y_vec_coarse[j + 1], coarse.z[i, j + 1, 0]),
                     (x_vec_coarse[i + 1], y_vec_coarse[j + 1], coarse.z[i + 1, j + 1, 0])]
            for ii in range(2*i, 2*i + 3):
                for jj in range(2*j, 2*j + 3):            
                    intz[ii][jj][0] = bilin(x_vec_fine[ii], y_vec_fine[jj], points)
                    errz_mtx[ii][jj][0] = abs(terrain.z[ii, jj, 0] - intz[ii][jj][0])
    
    errz = list(errz_mtx.flatten("C"))
    
    terrain['errors'] = pv.plotting.normalize(errz_mtx.flatten("C"))
    intmesh = copy.deepcopy(terrain)
    
    return (intz, errz, intmesh)
    
    ### END STUDENT SOLUTION

In [26]:
# x_vec = list(terrain.x[:-1,0,0])
# x_vec
# errz

In [27]:
### Tests for vizError. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.
intz, errz, intmesh = reconstructMesh(terrain,coarse)
np.testing.assert_allclose(intz[130][130][0],2547.8, rtol=1e-4)
np.testing.assert_allclose(intz[247][13][0],2142.71, rtol=1e-5)
np.testing.assert_allclose(errz[89],1.89996337890625, rtol=1e-2)
np.testing.assert_allclose(errz[30678],1.18499755859375, rtol=1e-2)
np.testing.assert_allclose(errz[-10],1.0299072265625, rtol=1e-2)

In [28]:
intmesh

StructuredGrid (0x7f987b558760)
  N Cells:	65536
  N Points:	66049
  X Bounds:	1.819e+06, 1.822e+06
  Y Bounds:	5.647e+06, 5.651e+06
  Z Bounds:	1.777e+03, 2.787e+03
  Dimensions:	257, 257, 1
  N Arrays:	2

Now we can visualize the error values that we computed! We recommend adjusting the color map to better visualize the error values. You can change the color map by clicking the settings icon at the top left of the interface.

In [29]:
print(f"Visualizing error between resolutions {terrain.dimensions[0]} and {math.ceil(terrain.dimensions[0]/2)}...")

pv.plot(intmesh, scalars='errors')

Visualizing error between resolutions 257 and 129...


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

For reference, here is a sample of what your final visualization should look like (with the magma colormap applied):
<img src='error-visualization.png' width=600/>

As you can see the error values are represented in thin lines. One of the things that can be done to visualize the erroneous regions in a clearer way is to smooth the image. There are different methods for image smoothing, but here we use a simple one that updates the error value of each pixel as the average of the error of its surrounding pixels. For this purpose we want to use a $k \times k$ 2D convolution with a value of $\frac{1}{k^2}$ at each element. `scipy.signal` has an implementation of this function. The `errz` list that your `reconstructMesh` outputs is $1$-dimensional, so you would have to reshape it to the correct dimensions of your grid before giving that as input to the `convolve2d` function. You need to flatten this array again before returning that in the function below. For `convolve2d`, use the symmetrical boundary condition (see this for more info: 
[convolve2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html))

In [30]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
#NOTE: You do not need to round any values within your results.
from scipy import signal

def smoothed_errors(errz, shape, kernel_size):
    ### BEGIN STUDENT SOLUTION
    if isinstance(errz, list):
        errz = np.array(errz)
    errz_mtx = errz.reshape(shape[0], shape[1])
    kernel = np.ones((kernel_size, kernel_size)) * (1 / kernel_size**2) 
    conv_2d = signal.convolve2d(errz_mtx, kernel, boundary='symm', mode='same')
    
    smoothed_errz = conv_2d.flatten("C")                                
    
    return smoothed_errz

    ### END STUDENT SOLUTION

In [31]:
shape = (terrain.z.shape[0], terrain.z.shape[1])
np.random.seed(1)
rand_errz = np.random.uniform(low=0.0, high=1.0, size=len(errz))
smoothed_errz = smoothed_errors(rand_errz, shape, 4)
smoothed_errz.shape

(66049,)

In [32]:
### Tests for smoothing the error. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.
shape = (terrain.z.shape[0], terrain.z.shape[1])
np.random.seed(1)
rand_errz = np.random.uniform(low=0.0, high=1.0, size=len(errz))
smoothed_errz = smoothed_errors(rand_errz, shape, 4)
np.testing.assert_allclose(smoothed_errz[95],0.5182084931710876, rtol=1e-2)
np.testing.assert_allclose(smoothed_errz[103],0.6119100598363368, rtol=1e-2)

Now we can use this function to smooth the error values before visualizing it.

In [33]:
smoothed_errz = smoothed_errors(errz, shape, 2)
intmesh['errors'] = pv.plotting.normalize(smoothed_errz)
pl = pv.plot(intmesh, scalars='errors')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

As we increase the kernel size, the error values become smoother and less informative about the specific regions with higher error values:

In [34]:
smoothed_errz = smoothed_errors(errz, shape, 5)
intmesh['errors'] = pv.plotting.normalize(smoothed_errz)
pl = pv.plot(intmesh, scalars='errors')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

One way to keep the regions with larger error values in such visualizations more noticeable is to apply a non-linear transformation to the error values. One such simple transformation is to raise all the values to the same power. If this value is larger than 1, the larger values increase more greatly compared to the smaller values. If the power is less than 1, the values become more similar to each other:

In [35]:
smoothed_errz = smoothed_errors(errz, shape, 5)
transformed_errz = smoothed_errz**2
intmesh['errors'] = pv.plotting.normalize(transformed_errz)
pl = pv.plot(intmesh, scalars='errors')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [36]:
smoothed_errz = smoothed_errors(errz, shape, 5)
transformed_errz = smoothed_errz**0.5
intmesh['errors'] = pv.plotting.normalize(transformed_errz)
pl = pv.plot(intmesh, scalars='errors')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…